Modèle sans convolution (réseau de neurones classique) sur le dataset du Joueur bas - Henri, Minh, Maxime
- Coup droit
- Déplacement
- Revers
- Service

In [4]:
import numpy as np
import cv2
import os
from smart.video import Video, Image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import tensorflow as tf
import random
from sklearn.preprocessing import LabelEncoder

In [5]:
# Configuration
path_dataset = 'img/JBas'

# Coup droit, revers, déplacement

## Prises des frames et des vidéos

In [2]:
list_videos = []

In [3]:
y = []
all_paths = []
for dirpath, dirnames, _ in os.walk(path_dataset):
    for dir_type in dirnames:
        
        path_fol = os.path.join(dirpath, dir_type)
        for file in os.listdir(path_fol):
            path_fol_img = os.path.join(path_fol, file)
            if os.path.isdir(path_fol_img) and file == 'images':
                frames = []
                for file_img in os.listdir(path_fol_img):
                    path_img = os.path.join(path_fol_img, file_img)
                    img_obj = Image.load_image(cv2.IMREAD_GRAYSCALE, path_img)
                    img = img_obj.img
                    frames.append(img)	
                if (len(frames) == 15):
                    output_res = path_img.split('\\')[1]
                    all_paths.append(path_fol_img)
                    vid = Video.read_video_from_frames(frames)
                    list_videos.append(vid)
                    y.append(output_res)

In [4]:
list_videos

In [5]:
# Output sans label encoder
y

['coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup dro

In [6]:
# One-hot encoding
encoder = LabelEncoder()

Transformer output y en one hot vector
- 0: coup droit
- 1: déplacement
- 2: revers
- 3: service

In [7]:
y = encoder.fit_transform(y)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int64)

In [8]:
output_y = np.unique(y)
output_y

array([0, 1, 2, 3], dtype=int64)

In [9]:
# Résolution des images
input_shape_model = list_videos[0].frames.shape[0] * list_videos[0].frames.shape[1] * list_videos[0].frames.shape[2]
input_shape_model

37500

In [10]:
len(list_videos)

214

In [11]:
train_size = int(len(list_videos) * 0.8)
test_size = len(list_videos) - train_size

In [12]:
## Split video train, test
random.seed(42)
vids_train = random.sample(list_videos, k = train_size)
vids_test = [vid for vid in list_videos if vid not in vids_train]
y_train = [y[i] for i in range(len(y)) if list_videos[i] in vids_train]
y_test = [y[i] for i in range(len(y)) if list_videos[i] in vids_test]
paths_test = [all_paths[i] for i in range(len(all_paths)) if list_videos[i] in vids_test]
len(vids_train), len(y_train), len(vids_test), len(y_test)

(171, 171, 43, 43)

In [13]:
paths_test

['img/JBas\\coup droit\\1980-2563441\\images',
 'img/JBas\\coup droit\\1980-2563523\\images',
 'img/JBas\\coup droit\\42698-4288095\\images',
 'img/JBas\\coup droit\\a36r805\\images',
 'img/JBas\\coup droit\\Alcaraz_Sinner270\\images',
 'img/JBas\\coup droit\\AS_13416\\images',
 'img/JBas\\coup droit\\AS_22373\\images',
 'img/JBas\\coup droit\\AS_30502\\images',
 'img/JBas\\coup droit\\AS_37288\\images',
 'img/JBas\\coup droit\\AS_40159\\images',
 'img/JBas\\coup droit\\car0001-0342253\\images',
 'img/JBas\\coup droit\\car5493-5847185\\images',
 'img/JBas\\coup droit\\na0201977\\images',
 'img/JBas\\coup droit\\uopc1041\\images',
 'img/JBas\\coup droit\\uopc1683\\images',
 'img/JBas\\coup droit\\uopc339\\images',
 'img/JBas\\deplacement\\a36r1216\\images',
 'img/JBas\\deplacement\\a36r1713\\images',
 'img/JBas\\deplacement\\a36r172\\images',
 'img/JBas\\deplacement\\a36r257\\images',
 'img/JBas\\deplacement\\a36r339\\images',
 'img/JBas\\deplacement\\AS_13292\\images',
 'img/JBas\\depl

In [14]:
y_train = np.array(y_train, dtype=int)
y_test = np.array(y_test, dtype=int)
y_train

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [15]:
# Split input X en train, test set
X_train = np.zeros((train_size, 50*50*15), dtype=int)
for i, vid in enumerate(vids_train):
    seq_img = vid.frames.flatten()
    X_train[i] = seq_img
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [16]:
X_test = np.zeros((test_size, 50*50*15), dtype=int)
for i, vid in enumerate(vids_test):
    seq_img = vid.frames.flatten()
    X_test[i] = seq_img
X_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [17]:
# Normalisation
X_train = X_train / 255.0
X_test = X_test / 255.0

## Modèle de réseau de neurones avec classification multiclass

In [18]:
tf.random.set_seed(1234) # for consistent results
model = Sequential(
    [               
        tf.keras.Input(shape = input_shape_model), # 50*50 * 15 = 37500
        Dense(units=16, activation="relu"),
        Dense(units=32, activation="relu"),
        Dense(units=64, activation="sigmoid"),
        Dense(units=128, activation="relu"),
        Dense(units=len(output_y), activation="softmax"),
    ]
)          
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                600016    
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dense_2 (Dense)             (None, 64)                2112      
                                                                 
 dense_3 (Dense)             (None, 128)               8320      
                                                                 
 dense_4 (Dense)             (None, 4)                 516       
                                                                 
Total params: 611,508
Trainable params: 611,508
Non-trainable params: 0
_________________________________________________________________


In [19]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [20]:
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = Adam(learning_rate=0.001), # 'Adam',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    epochs = 70,
    batch_size = 32, # données transmises pour une session
    validation_split=0.2,
    callbacks=callbacks_list
)

Epoch 1/70
1/5 [=====>........................] - ETA: 2s - loss: 1.4116 - accuracy: 0.3750
Epoch 1: val_loss improved from inf to 2.72903, saving model to Weights-001--2.72903.hdf5
5/5 [==============================] - 1s 103ms/step - loss: 1.2571 - accuracy: 0.3676 - val_loss: 2.7290 - val_accuracy: 0.0000e+00
Epoch 2/70
1/5 [=====>........................] - ETA: 0s - loss: 1.1106 - accuracy: 0.3438
Epoch 2: val_loss did not improve from 2.72903
5/5 [==============================] - 0s 20ms/step - loss: 1.0812 - accuracy: 0.4412 - val_loss: 3.6083 - val_accuracy: 0.0000e+00
Epoch 3/70
1/5 [=====>........................] - ETA: 0s - loss: 1.0516 - accuracy: 0.5312
Epoch 3: val_loss did not improve from 2.72903
5/5 [==============================] - 0s 20ms/step - loss: 1.0073 - accuracy: 0.5368 - val_loss: 4.1296 - val_accuracy: 0.0000e+00
Epoch 4/70
5/5 [==============================] - ETA: 0s - loss: 0.9411 - accuracy: 0.5588
Epoch 4: val_loss did not improve from 2.72903
5/5 

In [21]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 5ms/step - loss: 3.5778 - accuracy: 0.3953


[3.577824115753174, 0.39534884691238403]

In [22]:
pred = model.predict(X_test)
pred

2/2 [==============================] - 0s 3ms/step


array([[5.9259776e-03, 9.9274302e-01, 1.3035609e-03, 2.7502851e-05],
       [9.8601228e-01, 3.5098656e-03, 1.0430318e-02, 4.7538862e-05],
       [3.1918364e-03, 9.9658185e-01, 2.1220079e-04, 1.4194241e-05],
       [4.3782455e-04, 9.9905330e-01, 5.0205475e-04, 6.7542064e-06],
       [4.4629714e-04, 9.9936897e-01, 1.7952736e-04, 5.3009680e-06],
       [8.9080566e-01, 9.1820978e-02, 1.7214239e-02, 1.5915820e-04],
       [3.3088568e-01, 1.0922292e-01, 5.5952144e-01, 3.6999091e-04],
       [1.4510445e-01, 6.8976277e-01, 1.6481198e-01, 3.2078949e-04],
       [9.9739099e-01, 2.2334959e-03, 3.6017620e-04, 1.5349289e-05],
       [5.9411433e-03, 9.9375808e-01, 2.8281682e-04, 1.8024199e-05],
       [6.4807072e-02, 1.4824430e-02, 9.2023993e-01, 1.2864785e-04],
       [9.9934393e-01, 5.0754234e-04, 1.4158855e-04, 6.8307868e-06],
       [1.0224879e-02, 9.8798776e-01, 1.7530622e-03, 3.4275941e-05],
       [6.5341091e-01, 3.4440917e-01, 2.0555428e-03, 1.2445383e-04],
       [1.9753610e-01, 7.9048276e-

In [23]:
y_pred = np.argmax(pred, axis=1)
print("Predicted labels: ", y_pred)
print("True labels:      ", y_test)

Predicted labels:  [1 0 1 1 1 0 2 1 0 1 2 0 1 0 1 0 1 1 2 1 1 2 2 0 1 2 2 1 1 1 1 1 0 2 1 0 0
 0 1 0 1 0 0]
True labels:       [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 2 2 3 3 3 3]


**Accuracy: $17 / 43$ ($39.53 \%$)**

# Réseaux de neurones convolutifs

## Prises des frames et des vidéos

In [6]:
list_videos_cnn = []
y_cnn = []
for dirpath, dirnames, _ in os.walk(path_dataset):
    for dir_type in dirnames:
        
        path_fol = os.path.join(dirpath, dir_type)
        for file in os.listdir(path_fol):
            path_fol_img = os.path.join(path_fol, file)
            if os.path.isdir(path_fol_img) and file == 'images':
                frames = []
                for file_img in os.listdir(path_fol_img):
                    path_img = os.path.join(path_fol_img, file_img)
                    img_obj = Image.load_image(cv2.IMREAD_COLOR, path_img)
                    img = img_obj.img
                    frames.append(img)	
                if (len(frames) == 15):
                    output_res = path_img.split('\\')[1]
                    vid = Video.read_video_from_frames(frames)
                    list_videos_cnn.append(vid)
                    y_cnn.append(output_res)

In [7]:
list_videos_cnn[0].frames.shape[1:]

(50, 50, 3)

In [8]:
# One-hot encoding
encoder = LabelEncoder()

Transformer output y en one hot vector
- 0: coup droit
- 1: déplacement
- 2: revers
- 3: service

In [9]:
y_cnn = encoder.fit_transform(y_cnn)
y_cnn

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int64)

In [10]:
output_y = np.unique(y_cnn)
output_y

array([0, 1, 2, 3], dtype=int64)

In [11]:
len(list_videos_cnn)

214

In [12]:
train_size = int(len(list_videos_cnn) * 0.8)
test_size = len(list_videos_cnn) - train_size

In [13]:
## Split video train, test
random.seed(42)
vids_train_cnn = random.sample(list_videos_cnn, k = train_size)
vids_test_cnn = [vid for vid in list_videos_cnn if vid not in vids_train_cnn]
y_train_cnn = [y_cnn[i] for i in range(len(y_cnn)) if list_videos_cnn[i] in vids_train_cnn]
y_test_cnn = [y_cnn[i] for i in range(len(y_cnn)) if list_videos_cnn[i] in vids_test_cnn]
len(vids_train_cnn), len(y_train_cnn), len(vids_test_cnn), len(y_test_cnn)

(171, 171, 43, 43)

In [14]:
y_train_cnn = np.array(y_train_cnn, dtype=int)
y_test_cnn = np.array(y_test_cnn, dtype=int)
y_train_cnn

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [15]:
# Split input X en train, test set
X_train_cnn = np.zeros((train_size, 50, 15*50, 3), dtype=int)
for i, vid in enumerate(vids_train_cnn):
    for j, frame in enumerate(vid.frames):
        X_train_cnn[i, :, 50*j:50*(j+1), :] = frame
X_train_cnn.shape

(171, 50, 750, 3)

In [16]:
X_test_cnn = np.zeros((test_size, 50, 15*50, 3), dtype=int)
for i, vid in enumerate(vids_test_cnn):
    for j, frame in enumerate(vid.frames):
        X_test_cnn[i, :, 50*j:50*(j+1), :] = frame
X_test_cnn.shape

(43, 50, 750, 3)

In [17]:
input_shape_cnn = X_train_cnn.shape[1:]
input_shape_cnn

(50, 750, 3)

## Modèle de réseau de neurones convolutifs

In [18]:
model_cnn = Sequential(
    [
        tf.keras.Input(shape = input_shape_cnn), # 50, 15*50, 3
        Conv2D(filters=16, kernel_size=(3, 3), activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        Conv2D(filters=32, kernel_size=(3, 3), activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        Conv2D(filters=64, kernel_size=(3, 3), activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        Flatten(),
        Dense(units=128, activation="relu"),
        Dense(units=len(output_y), activation="softmax"),
    ]
)

model_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 748, 16)       448       
                                                                 
 batch_normalization (BatchN  (None, 48, 748, 16)      64        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 374, 16)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 374, 16)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 372, 32)       4640      
                                                                 
 batch_normalization_1 (Batc  (None, 22, 372, 32)      1

In [19]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [20]:
model_cnn.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer='rmsprop',
    metrics=['accuracy']
)

history_cnn = model_cnn.fit(
    X_train_cnn, y_train_cnn,
    epochs = 70,
    batch_size = 32, # données transmises pour une session
    validation_split=0.2,
    callbacks=callbacks_list
)

Epoch 1/70
5/5 [==============================] - ETA: 0s - loss: 53.1491 - accuracy: 0.3309
Epoch 1: val_loss improved from inf to 642.20526, saving model to Weights-001--642.20526.hdf5
5/5 [==============================] - 5s 734ms/step - loss: 53.1491 - accuracy: 0.3309 - val_loss: 642.2053 - val_accuracy: 0.0000e+00
Epoch 2/70
5/5 [==============================] - ETA: 0s - loss: 13.9353 - accuracy: 0.3897
Epoch 2: val_loss improved from 642.20526 to 295.24750, saving model to Weights-002--295.24750.hdf5
5/5 [==============================] - 3s 549ms/step - loss: 13.9353 - accuracy: 0.3897 - val_loss: 295.2475 - val_accuracy: 0.2857
Epoch 3/70
5/5 [==============================] - ETA: 0s - loss: 11.5706 - accuracy: 0.4485
Epoch 3: val_loss improved from 295.24750 to 175.12643, saving model to Weights-003--175.12643.hdf5
5/5 [==============================] - 3s 658ms/step - loss: 11.5706 - accuracy: 0.4485 - val_loss: 175.1264 - val_accuracy: 0.2571
Epoch 4/70
5/5 [===========

In [24]:
model_cnn.evaluate(X_test_cnn, y_test_cnn)

2/2 [==============================] - 1s 183ms/step - loss: 5.8162 - accuracy: 0.4419


[5.81617546081543, 0.44186046719551086]

In [25]:
pred_cnn = model_cnn.predict(X_test_cnn)
pred_cnn

2/2 [==============================] - 1s 146ms/step


array([[5.5714085e-05, 9.9989319e-01, 5.0722883e-05, 3.0792046e-07],
       [9.9980098e-01, 4.1563744e-05, 1.5749715e-04, 7.6666586e-13],
       [7.5838739e-01, 2.4161255e-01, 2.0436968e-08, 8.3754607e-11],
       [1.3874187e-03, 9.8977822e-01, 8.8343536e-03, 9.9004506e-11],
       [1.9859394e-06, 9.9995410e-01, 4.3856402e-05, 1.0857977e-12],
       [1.8935978e-01, 6.7827404e-01, 1.3236614e-01, 1.3408807e-14],
       [6.5355822e-02, 1.0895499e-02, 9.2374861e-01, 8.3503695e-13],
       [9.9958664e-01, 4.1338801e-04, 2.0195272e-09, 7.9843785e-14],
       [9.9995959e-01, 4.0467839e-05, 8.1417423e-10, 4.2615612e-16],
       [6.8663359e-03, 9.9313354e-01, 6.3140554e-08, 1.9713835e-10],
       [2.7514318e-01, 2.9292309e-03, 7.2192758e-01, 4.6424008e-13],
       [9.9976844e-01, 1.3347029e-06, 2.3028457e-04, 8.6614462e-18],
       [2.3402955e-02, 8.8485008e-01, 9.1746084e-02, 8.3658222e-07],
       [8.8203526e-01, 1.1795700e-01, 7.7017294e-06, 2.2379727e-14],
       [9.5957547e-01, 3.9123803e-

In [26]:
y_pred_cnn = np.argmax(pred_cnn, axis=1)
print("Predicted labels: ", y_pred_cnn)
print("True labels:      ", y_test_cnn)

Predicted labels:  [1 0 0 1 1 1 2 0 0 1 2 0 1 0 0 2 1 1 1 1 1 0 2 0 1 1 1 1 2 1 1 1 0 1 1 0 0
 0 1 0 1 0 0]
True labels:       [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2
 2 2 3 3 3 3]


**Accuracy: $19/43$ ($44.18 \%$)**